In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/CHBAM/.secret/yelp_api.json') as f:
    login = json.load(f)
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [11]:
# set our API call parameters 
LOCATION = 'TX,TX'
TERM = 'Lasagna'

In [12]:
JSON_FILE = "Data/results_tx_Lasagna.json"
folder = os.path.dirname(JSON_FILE)
## If JSON_FILE included a folder:
if len(folder)>0:
   # create the folder
   os.makedirs(folder,exist_ok=True)
   
## INFORM USER AND SAVE EMPTY LIST
print(f'[i] {JSON_FILE} not found. Saving empty list to file.')


# save an empty list
with open(JSON_FILE,'w') as f:
   json.dump([],f)  

[i] Data/results_tx_Lasagna.json not found. Saving empty list to file.


In [13]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                                offset=0)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [14]:
total_results = results['total']
total_results

547

In [15]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [16]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-0)/ results_per_page)
n_pages

28

In [17]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/28 [00:00<?, ?it/s]

In [18]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    try:
      results = yelp_api.search_query(location=LOCATION,
                                       term=TERM, 
                                       offset=n_results)
    except:
        print('Error encountered on :' , results)
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 300ms pause
    time.sleep(.3)

  0%|          | 0/28 [00:00<?, ?it/s]

In [19]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,McwaN30rgpfIpyDPT913Vw,fabios-artisan-pasta-houston,Fabio's Artisan Pasta,https://s3-media1.fl.yelpcdn.com/bphoto/Um6Q53...,False,https://www.yelp.com/biz/fabios-artisan-pasta-...,323,"[{'alias': 'pastashops', 'title': 'Pasta Shops'}]",5.0,"{'latitude': 29.73823, 'longitude': -95.41208}",[],$$,"{'address1': '2129 W Alabama St', 'address2': ...",+17135281329,(713) 528-1329,5740.673231
1,GAujfOsl26j_BcGLU0Ir5g,bellissimo-houston,Bellissimo,https://s3-media2.fl.yelpcdn.com/bphoto/9wsPmY...,False,https://www.yelp.com/biz/bellissimo-houston?ad...,54,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 29.81634, 'longitude': -95.38732}","[pickup, delivery]",NaN,"{'address1': '902 E 33rd St', 'address2': '', ...",+18326181168,(832) 618-1168,6489.632602
2,k8wZ3wCkCRJmVoPVL80CMQ,lasagna-house-iii-houston-2,Lasagna House III,https://s3-media1.fl.yelpcdn.com/bphoto/X444MT...,False,https://www.yelp.com/biz/lasagna-house-iii-hou...,288,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 29.7371678493099, 'longitude': -9...",[delivery],$$,"{'address1': '13306 Westheimer Rd', 'address2'...",+12814969114,(281) 496-9114,25717.447239
3,KDkl-gEvbOyEYzuI6JNAlA,mandolas-deli-houston,Mandola's Deli,https://s3-media4.fl.yelpcdn.com/bphoto/Qluxqp...,False,https://www.yelp.com/biz/mandolas-deli-houston...,336,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 29.73755, 'longitude': -95.33926}",[delivery],$,"{'address1': '4105 Leeland St', 'address2': ''...",+17132235186,(713) 223-5186,3452.877569
4,x4WealiPVG-tzwzhe7nvpw,north-italia-houston,North Italia,https://s3-media3.fl.yelpcdn.com/bphoto/jNVzNb...,False,https://www.yelp.com/biz/north-italia-houston?...,2279,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 29.7485868638914, 'longitude': -9...",[delivery],$$,"{'address1': '1700 Post Oak Blvd', 'address2':...",+12816054030,(281) 605-4030,9983.179391


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
540,-yzDKjByXsyVYN5Z2Fk3Hw,dennys-spring-5,Denny's,https://s3-media2.fl.yelpcdn.com/bphoto/LI3NV0...,False,https://www.yelp.com/biz/dennys-spring-5?adjus...,80,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",3.0,"{'latitude': 30.0542536148866, 'longitude': -9...","[delivery, pickup]",$,"{'address1': '20015 I-45 N', 'address2': '', '...",+12813506184,(281) 350-6184,33143.412570
541,egmNNzOZxQY_aqt_5bLOVQ,dennys-sugar-land,Denny's,https://s3-media1.fl.yelpcdn.com/bphoto/D4OHH2...,False,https://www.yelp.com/biz/dennys-sugar-land?adj...,110,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",2.5,"{'latitude': 29.601682104609, 'longitude': -95...","[delivery, pickup]",$,"{'address1': '1422 State Hwy 6 S', 'address2':...",+12812400327,(281) 240-0327,31909.978663
542,UymA6289ijCFiU7kvMAlFg,dennys-spring-7,Denny's,https://s3-media1.fl.yelpcdn.com/bphoto/SWah7u...,False,https://www.yelp.com/biz/dennys-spring-7?adjus...,53,"[{'alias': 'tradamerican', 'title': 'American'...",3.0,"{'latitude': 30.0745147, 'longitude': -95.5232...","[delivery, pickup]",$,"{'address1': '6504 Fm 2920', 'address2': '', '...",+12813792725,(281) 379-2725,38044.271493
543,Pmin3vyANl_UBU4SY-H54g,golden-corral-buffet-and-grill-humble-2,Golden Corral Buffet & Grill,https://s3-media4.fl.yelpcdn.com/bphoto/xaHCO1...,False,https://www.yelp.com/biz/golden-corral-buffet-...,81,"[{'alias': 'buffets', 'title': 'Buffets'}, {'a...",2.0,"{'latitude': 30.0055813804803, 'longitude': -9...",[delivery],$$,"{'address1': '9663 Fm 1960 Bypass', 'address2'...",+12815401051,(281) 540-1051,28103.357838
544,dNYJtjtZlqX0lPWwp-9ybg,pizza-hut-cypress-3,Pizza Hut,https://s3-media4.fl.yelpcdn.com/bphoto/J3fG_B...,False,https://www.yelp.com/biz/pizza-hut-cypress-3?a...,56,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",1.5,"{'latitude': 29.98314459530259, 'longitude': -...",[delivery],$,"{'address1': '13750 N Eldridge Pkwy', 'address...",+12812576172,(281) 257-6172,34414.259910


In [20]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [21]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_tx_lasagna.csv.gz', compression='gzip',index=False)